<a href="https://colab.research.google.com/github/kartheek274/Python/blob/main/Churn_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

In [17]:
data = pd.read_csv('/content/sample_data/churn_modelling.csv')
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [26]:
from sklearn.preprocessing import LabelEncoder
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany,Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [19]:
from sklearn.preprocessing import OneHotEncoder
ohe_geo = OneHotEncoder()
geo = ohe_geo.fit_transform(data[['Geography']]).toarray()
geo = pd.DataFrame(geo, columns=['France', 'Germany', 'Spain'])
geo

,France,Germany,Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [20]:
data = data.drop(['Geography', 'RowNumber', 'CustomerId', 'Surname'], axis=1)


In [21]:
data = pd.concat([data, geo], axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany,Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [22]:
from sklearn.model_selection import train_test_split
x = data.drop(['Exited'], axis=1)
y = data['Exited']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [30]:
from sklearn.preprocessing  import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [31]:
import pickle
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('ohe_geo.pkl', 'wb') as file:
    pickle.dump(ohe_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [35]:
# Define a Model
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, TensorBoard
model = Sequential([
      Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
      Dense(32, activation='relu'),
      Dense(1, activation='sigmoid')
]
)


In [37]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [44]:
import tensorflow
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate= 0.01)
loss = "binary_crossentropy"
opt

In [43]:
model.compile(optimizer = opt,loss = "binary_crossentropy",metrics = ["accuracy"])

In [47]:
import datetime
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq =1)
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights= True)

In [48]:
history  = model.fit(
    x_train,y_train,validation_data= (x_test, y_test),epochs =100,
    callbacks=[tensorflow_callback, early_stopping_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8760 - loss: 0.2812 - val_accuracy: 0.8565 - val_loss: 0.3694
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8835 - loss: 0.2708 - val_accuracy: 0.8510 - val_loss: 0.3968
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8786 - loss: 0.2786 - val_accuracy: 0.8555 - val_loss: 0.4046
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8834 - loss: 0.2766 - val_accuracy: 0.8475 - val_loss: 0.3901
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8851 - loss: 0.2631 - val_accuracy: 0.8490 - val_loss: 0.3773
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8846 - loss: 0.2688 - val_accuracy: 0.8475 - val_loss: 0.4103


In [49]:
model.save('model.h5')

In [132]:
input_data = {
    'CreditScore': 200,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 600,
    'NumOfProducts': 1,
    'HasCrCard': 0,
    'IsActiveMember': 0,
    'EstimatedSalary': 500

}

In [133]:
geo_encoded = ohe_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded = pd.DataFrame(geo_encoded, columns=ohe_geo.get_feature_names_out(['Geography']))
geo_encoded


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [134]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,200,France,Male,40,3,600,1,0,0,500


In [135]:
import pickle
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Define a new LabelEncoder and fit it with the known classes
# This is a diagnostic step to rule out issues with the pickled object
label_encoder_gender = LabelEncoder()
label_encoder_gender.fit(['Female', 'Male']) # Fit with the expected string values

print("Newly created LabelEncoder classes:", label_encoder_gender.classes_)

# Get the gender value as a string
gender_value_str = str(input_df['Gender'].iloc[0])

# Prepare the input for transformation as a NumPy array of strings, reshaped to be 2D
# Use the NumPy array for transformation
gender_input_for_transform = np.array([gender_value_str]).reshape(-1, 1)

input_df['Gender'] = label_encoder_gender.transform(gender_input_for_transform)

input_df

Newly created LabelEncoder classes: ['Female' 'Male']


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,200,France,1,40,3,600,1,0,0,500


In [136]:
input_df = pd.concat([input_df.drop("Geography",axis =1), geo_encoded], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,200,1,40,3,600,1,0,0,500,1.0,0.0,0.0


In [137]:
input_scaled = scaler.transform(input_df)
input_scaled

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


array([[ 2.00000000e+02,  1.00000000e+00,  4.00000000e+01,
         3.00000000e+00,  6.00000000e+02,  1.00000000e+00,
        -4.08562073e-17,  1.77635684e-17,  5.00000000e+02,
         1.00000000e+00,  1.23456800e-16, -1.42108547e-17]])

In [138]:
prediction = model.predict(input_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step


In [139]:
prediction_proba = prediction[0][0]
prediction_proba

np.float32(1.0)

In [140]:
if prediction_proba > 0.5:
  print("customer likely to churn")
else:
  print("customer is not likely to churn")

customer likely to churn
